In [10]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder, StandardScaler, KBinsDiscretizer
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

In [2]:
def data_collection(filepath):

    # lê o dataset
    df = pd.read_csv(filepath)

    # monta um dataframe com todas as features a porcentagem dos dados nulos
    null_data = pd.DataFrame(data = [list(df.columns), list(df.isnull().sum())]).transpose()
    null_data.columns = ['feature', 'null_data']

    # filtra o dataframe para colunas com no máximo 50% dos dados ausentes
    df = df.filter(items = null_data.feature.loc[null_data.null_data < 0.5 * df.shape[0]])

    # exclui as colunas que não serão usadas na análise
    df.drop(columns = ['Unnamed: 0', 'fl_matriz','natureza_juridica_macro','de_ramo','fl_spa', 'fl_antt',
    'idade_empresa_anos','vl_total_veiculos_pesados_grupo','vl_total_veiculos_leves_grupo','fl_veiculo',
    'fl_me','fl_sa','fl_epp','fl_mei','fl_ltda','dt_situacao','fl_st_especial','fl_email','fl_telefone',
    'fl_rm','nm_divisao','fl_optante_simples','sg_uf_matriz','de_saude_tributaria','de_saude_rescencia',
    'nu_meses_rescencia','fl_simples_irregular','empsetorcensitariofaixarendapopulacao','nm_meso_regiao',
    'nm_micro_regiao','fl_passivel_iss','idade_media_socios','idade_maxima_socios','idade_minima_socios',
    'qt_socios_st_regular','de_faixa_faturamento_estimado','vl_faturamento_estimado_grupo_aux',
    'vl_faturamento_estimado_aux','qt_socios','qt_socios_pj','qt_socios_pf', 'qt_filiais'], axis = 1, inplace = True)

    return df

In [3]:
def data_preprocessing(df):

    # todos os setores que estão nulos serão classificados na nova categoria OUTROS
    df.setor.fillna('OUTROS', inplace = True)
    # todos os segmentos que estão nulos serão classificados na nova categoria OUTROS
    df.nm_segmento.fillna('OUTROS', inplace =  True)
    # para os dados que estiverem com dado nulo será inserido a categoria que mais se repete
    df.de_faixa_faturamento_estimado_grupo.fillna(df.de_faixa_faturamento_estimado_grupo.mode().values[0], inplace = True)
    # para os dados que estiverem com dado nulo será inserido a categoria que mais se repete, porém será realizado um group 
    # by a partir do nível de ativade da empresa
    faixa_faturamento_por_nivel = df.groupby(['de_faixa_faturamento_estimado_grupo'])['de_nivel_atividade'].agg(pd.Series.mode)
    df.de_nivel_atividade.fillna(df.de_faixa_faturamento_estimado_grupo.map(faixa_faturamento_por_nivel), inplace = True)

    # realiza o filtro para selecionar somente as features que serão utilizadas no modelo
    df = df.filter(items = ['id','de_natureza_juridica','sg_uf', 'setor', 'nm_segmento', 'idade_emp_cat','de_nivel_atividade',
    'de_faixa_faturamento_estimado_grupo'])

    # transforma as variáveis categóricas em variáveis discretas
    labelencoder = LabelEncoder()
    df.de_natureza_juridica = labelencoder.fit_transform(df.de_natureza_juridica)
    df.sg_uf = labelencoder.fit_transform(df.sg_uf)
    df.setor = labelencoder.fit_transform(df.setor)
    df.nm_segmento = labelencoder.fit_transform(df.nm_segmento)
    df.idade_emp_cat = labelencoder.fit_transform(df.idade_emp_cat)
    df.de_nivel_atividade = labelencoder.fit_transform(df.de_nivel_atividade)
    df.de_faixa_faturamento_estimado_grupo = labelencoder.fit_transform(df.de_faixa_faturamento_estimado_grupo)

    # padroniza os dados na mesma escala
    scaler = StandardScaler()
    ids = df.id
    features = df.filter(items = ['de_natureza_juridica','sg_uf', 'setor', 'nm_segmento', 'idade_emp_cat','de_nivel_atividade',
    'de_faixa_faturamento_estimado_grupo'])
    features = scaler.fit_transform(features)

    return df, ids, features

In [4]:
df = data_collection('../../estaticos_market.csv')

In [5]:
df, ids, features = data_preprocessing(df)

In [6]:
pca = PCA(n_components= 3)

In [7]:
features = pca.fit_transform(features)
features

array([[ 1.68947344,  0.80900402,  1.91533384],
       [ 0.37345542,  0.42263575, -0.14431578],
       [-2.01234179,  1.56140272, -0.21710243],
       ...,
       [-2.99492117, -0.2697872 , -0.69827926],
       [ 1.8391228 ,  0.2084492 ,  1.73378959],
       [-2.60201706,  0.27272993,  0.83134327]])

In [8]:
f1 = features[:1000,0]
f2 = features[:1000,1]
f3 = features[:1000,2]

In [17]:
fig = px.scatter_3d(x = f1, y = f2, z = f3, size_max = 0.7)
fig.show()